---
draft: true
---

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('GeocodedFakeIndividualConstituents.csv')
df.head()

,Postcode,Latitude,Longitude,NumberDonations,TotalDonated,AverageDonated,Newsletter
0,NG9 3WF,52.930121,-1.198353,4,61,15.25,1
1,NG9 4WP,52.921587,-1.247504,1,23,23.00,0
2,NG9 3EL,52.938985,-1.239510,1,30,30.00,0
3,NG1 9FH,52.955008,-1.141045,5,75,15.00,1
4,NG5 6QZ,52.996670,-1.106307,1,15,15.00,0


In [3]:
data = pd.read_csv('PCD_OA21_LSOA21_MSOA21_LTLA22_UTLA22_CAUTH22_NOV23_UK_LU_v2.csv', low_memory=False)
data = data[['pcds', 'lsoa21cd']]
data = data.rename({'pcds':'Postcode', 'lsoa21cd':'LSOA21'}, axis=1)
data.head()

,Postcode,LSOA21
0,AB1 0AA,S01006514
1,AB1 0AB,S01006514
2,AB1 0AD,S01006514
3,AB1 0AE,S01006853
4,AB1 0AF,S01006511


In [4]:
df = df.merge(data, on='Postcode', how='left')
df.sample(5)

,Postcode,Latitude,Longitude,NumberDonations,TotalDonated,AverageDonated,Newsletter,LSOA21
14,NG3 7GN,52.961606,-1.104514,2,30,15.0,0,E01013914
61,NG1 1EQ,52.955571,-1.143074,9,135,15.0,1,E01035500
79,NG7 3PB,52.956008,-1.171405,1,68,68.0,1,E01035501
62,NG6 9LE,52.997481,-1.188002,1,16,16.0,0,E01013889
78,NG5 1JG,52.979984,-1.174297,1,28,28.0,0,E01013839


In [5]:
# Load the LSOA lookup data
lookup_data = pd.read_csv('LSOA_(2011)_to_LSOA_(2021)_to_Local_Authority_District_(2022)_Lookup_for_England_and_Wales_(Version_2).csv')
lookup_data = lookup_data[['LSOA21CD','LSOA11CD']].rename({'LSOA21CD':'LSOA21', 'LSOA11CD':'LSOA11'}, axis=1)

# Load the IMD data
imd_data = pd.read_csv('File_7_-_All_IoD2019_Scores__Ranks__Deciles_and_Population_Denominators_3.csv')
imd_data = imd_data[['LSOA code (2011)', 'Index of Multiple Deprivation (IMD) Score']].rename({'LSOA code (2011)':'LSOA11', 'Index of Multiple Deprivation (IMD) Score':'IMD_Score'}, axis=1)

# Merge the IMD data with the lookup data
imd_data = imd_data.merge(lookup_data, on='LSOA11', how='left')

# Aggregate the IMD scores for each LSOA21
imd_data = imd_data.groupby('LSOA21')['IMD_Score'].mean().reset_index()

# Now merge your original dataframe with the aggregated IMD data
df = df.merge(imd_data, on='LSOA21', how='left')

In [6]:
df.sample(10)

,Postcode,Latitude,Longitude,NumberDonations,TotalDonated,AverageDonated,Newsletter,LSOA21,IMD_Score
21,NG9 2UB,52.931034,-1.211308,2,30,15.00,0,E01028079,9.134
60,NG8 6DQ,52.977857,-1.238111,3,45,15.00,1,E01013864,45.803
37,NG8 3DJ,52.970564,-1.219671,4,60,15.00,1,E01013867,49.935
66,NG11 7ES,52.922035,-1.164809,2,30,15.00,0,E01013902,11.793
16,NG1 3NE,52.958944,-1.147101,1,15,15.00,0,E01013961,35.376
43,NG3 6NY,52.968191,-1.114652,1,17,17.00,0,E01028161,18.522
15,NG1 1NR,52.953058,-1.144924,1,15,15.00,0,E01035506,24.712
4,NG5 6QZ,52.996670,-1.106307,1,15,15.00,0,E01028179,9.485
12,NG1 3RD,52.957021,-1.148493,14,213,15.21,1,E01033399,33.633
74,NG1 3JP,52.959868,-1.149375,8,16618,2077.25,1,E01013961,35.376


In [7]:
Pop_density_lookup = pd.read_csv('sape23dt11mid2020lsoapopulationdensity 4(Sheet1).csv')
Pop_density_lookup = Pop_density_lookup[['LSOA Code','People per Sq Km']].rename({'LSOA Code':'LSOA21', 
                                                                                  'People per Sq Km':'PopDensity'},axis=1)
df = df.merge(Pop_density_lookup, on='LSOA21', how='left')
df.head()

,Postcode,Latitude,Longitude,NumberDonations,TotalDonated,AverageDonated,Newsletter,LSOA21,IMD_Score,PopDensity
0,NG9 3WF,52.930121,-1.198353,4,61,15.25,1,E01028074,16.137,"5,866"
1,NG9 4WP,52.921587,-1.247504,1,23,23.00,0,E01028101,36.690,"3,140"
2,NG9 3EL,52.938985,-1.239510,1,30,30.00,0,E01028091,4.834,"3,194"
3,NG1 9FH,52.955008,-1.141045,5,75,15.00,1,E01035500,33.093,NaN
4,NG5 6QZ,52.996670,-1.106307,1,15,15.00,0,E01028179,9.485,"5,590"


In [8]:
df['PopDensity'] = df['PopDensity'].str.replace(',', '').astype(float)
df

,Postcode,Latitude,Longitude,NumberDonations,TotalDonated,AverageDonated,Newsletter,LSOA21,IMD_Score,PopDensity
0,NG9 3WF,52.930121,-1.198353,4,61,15.25,1,E01028074,16.137,5866.0
1,NG9 4WP,52.921587,-1.247504,1,23,23.00,0,E01028101,36.690,3140.0
2,NG9 3EL,52.938985,-1.239510,1,30,30.00,0,E01028091,4.834,3194.0
3,NG1 9FH,52.955008,-1.141045,5,75,15.00,1,E01035500,33.093,NaN
4,NG5 6QZ,52.996670,-1.106307,1,15,15.00,0,E01028179,9.485,5590.0
...,...,...,...,...,...,...,...,...,...,...
95,NG2 1WY,52.945107,-1.135586,1,15,15.00,0,E01033402,22.711,3290.0
96,NG8 1ND,52.958858,-1.196705,10,169,16.90,1,E01013978,17.118,6978.0
97,NG9 2QA,52.930121,-1.198353,1,15,15.00,0,E01028074,16.137,5866.0
98,NG3 1FF,52.958515,-1.144966,22,333,15.14,1,E01033399,33.633,21099.0


In [9]:
df.isna().sum()

Postcode            0
Latitude            0
Longitude           0
NumberDonations     0
TotalDonated        0
AverageDonated      0
Newsletter          0
LSOA21              0
IMD_Score           0
PopDensity         19
dtype: int64

In [10]:
df[df['PopDensity'].isnull()]

,Postcode,Latitude,Longitude,NumberDonations,TotalDonated,AverageDonated,Newsletter,LSOA21,IMD_Score,PopDensity
3,NG1 9FH,52.955008,-1.141045,5,75,15.00,1,E01035500,33.093,NaN
6,NG7 2FT,52.942328,-1.182442,1,15,15.00,0,E01035505,24.647,NaN
15,NG1 1NR,52.953058,-1.144924,1,15,15.00,0,E01035506,24.712,NaN
17,NG7 2GQ,52.948837,-1.180888,3,45,15.00,1,E01035507,17.973,NaN
26,NG3 3EN,52.955053,-1.141030,1,38,38.00,0,E01035500,33.093,NaN
34,NG1 1LF,52.955008,-1.141045,3,46,15.33,1,E01035500,33.093,NaN
35,NG1 1NN,52.953445,-1.144602,7,313,44.71,1,E01035506,24.712,NaN
42,NG3 2EU,52.956046,-1.141665,2,30,15.00,0,E01035500,33.093,NaN
53,NG7 1RA,52.951125,-1.172781,4,63,15.75,1,E01035505,24.647,NaN
61,NG1 1EQ,52.955571,-1.143074,9,135,15.00,1,E01035500,33.093,NaN


In [11]:
from sklearn.impute import KNNImputer

In [12]:
# Separate out the points with null PopDensity
df_null = df[df['PopDensity'].isnull()]

# Select only the columns that you want to impute
df_to_impute = df[['Latitude', 'Longitude', 'PopDensity']]

# Initialize KNNImputer
imputer = KNNImputer(n_neighbors=5, weights='distance')

# Perform the imputation
df_imputed_values = imputer.fit_transform(df_to_impute)

# Convert the result back to a DataFrame
df_imputed = pd.DataFrame(df_imputed_values, columns=[['Latitude', 'Longitude', 'PopDensity']])

# Replace the original PopDensity column with the imputed one
df['PopDensity'] = df_imputed['PopDensity']

# Get the imputed values for the points that were null
df_null_imputed = df.loc[df_null.index]

In [14]:
df_null_imputed

,Postcode,Latitude,Longitude,NumberDonations,TotalDonated,AverageDonated,Newsletter,LSOA21,IMD_Score,PopDensity
3,NG1 9FH,52.955008,-1.141045,5,75,15.00,1,E01035500,33.093,17808.154253
6,NG7 2FT,52.942328,-1.182442,1,15,15.00,0,E01035505,24.647,3378.092153
15,NG1 1NR,52.953058,-1.144924,1,15,15.00,0,E01035506,24.712,19180.230439
17,NG7 2GQ,52.948837,-1.180888,3,45,15.00,1,E01035507,17.973,6685.700513
26,NG3 3EN,52.955053,-1.141030,1,38,38.00,0,E01035500,33.093,17719.576839
34,NG1 1LF,52.955008,-1.141045,3,46,15.33,1,E01035500,33.093,17808.154253
35,NG1 1NN,52.953445,-1.144602,7,313,44.71,1,E01035506,24.712,19164.078011
42,NG3 2EU,52.956046,-1.141665,2,30,15.00,0,E01035500,33.093,15973.902634
53,NG7 1RA,52.951125,-1.172781,4,63,15.75,1,E01035505,24.647,9509.513045
61,NG1 1EQ,52.955571,-1.143074,9,135,15.00,1,E01035500,33.093,16475.142642


In [13]:
df.isna().sum()

Postcode           0
Latitude           0
Longitude          0
NumberDonations    0
TotalDonated       0
AverageDonated     0
Newsletter         0
LSOA21             0
IMD_Score          0
PopDensity         0
dtype: int64